In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import re
import gc
from sklearn.metrics import mean_squared_error, roc_auc_score

from xgboost import XGBClassifier, XGBRFRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from lightgbm import LGBMModel,LGBMClassifier
import copy

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-2idk07b7 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
from sklearn.feature_selection import RFE

# clf = SVC(kernel='sigmoid', random_state=42, probability=True, C=0.1, class_weight='balanced')
clf = LogisticRegression(solver='sag', random_state=42, class_weight='balanced')
# clf = XGBClassifier(
#         random_state = 42,
#         objective='binary:logistic',
#         n_estimators=500,
#         learning_rate=1e-3,
#         max_depth=3,
#         subsample=0.7,
#         scale_pos_weight=np.unique(y_train, return_counts=True)[1][0] / np.unique(y_train, return_counts=True)[1][1]
#     )
    

val_evals = []
aucs = []
f1s = []
for fidx in range(5):
    X_train = np.load('/work/zg78/som_folds/CSF_X_train_{}.npy'.format(fidx))
    y_train = np.load('/work/zg78/som_folds/CSF_y_train_{}.npy'.format(fidx))
    X_val = np.load('/work/zg78/som_folds/CSF_X_val_{}.npy'.format(fidx))
    y_val = np.load('/work/zg78/som_folds/CSF_y_val_{}.npy'.format(fidx))
    
    print(X_train.shape, y_train.shape, np.unique(y_val, return_counts=True))
    
    std_scaler = StandardScaler()
    std_scaler.fit(X_train)
    X_train = std_scaler.transform(X_train)
    X_val = std_scaler.transform(X_val)
        
    clf.fit(X_train, y_train)
    
    train_acc = metrics.accuracy_score(y_train, clf.predict(X_train))
    val_acc = metrics.accuracy_score(y_val, clf.predict(X_val))
    val_proba = clf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    f1 = metrics.f1_score(y_val, clf.predict(X_val))
    val_evals.append(val_acc)
    aucs.append(auc)
    f1s.append(f1)
    print(train_acc, val_acc, auc, f1, '\n')
    
print(sum(val_evals) / len(val_evals))
print(sum(aucs) / len(aucs))
print(sum(f1s) / len(f1s), np.std(f1s))
    
    

(60, 10) (60,) (array([0, 1]), array([12,  4]))
0.7833333333333333 0.6875 0.8125 0.6153846153846153 

(61, 10) (61,) (array([0, 1]), array([8, 7]))
0.8032786885245902 0.6 0.46428571428571425 0.5714285714285714 

(61, 10) (61,) (array([0, 1]), array([10,  5]))
0.7213114754098361 0.6666666666666666 0.74 0.4444444444444445 

(61, 10) (61,) (array([0, 1]), array([10,  5]))
0.7704918032786885 0.7333333333333333 0.7000000000000001 0.6 

(61, 10) (61,) (array([0, 1]), array([10,  5]))
0.7868852459016393 0.7333333333333333 0.8 0.6 

0.6841666666666667
0.7033571428571429
0.5662515262515262 0.06253918193194598


In [3]:
from sklearn.feature_selection import RFE

# clf = SVC(kernel='sigmoid', random_state=42, probability=True, C=0.1, class_weight='balanced')
clf = LogisticRegression(solver='sag', random_state=42, class_weight='balanced')
# clf = XGBClassifier(
#         random_state = 42,
#         objective='binary:logistic',
#         n_estimators=500,
#         learning_rate=1e-3,
#         max_depth=3,
#         subsample=0.7,
#         scale_pos_weight=np.unique(y_train, return_counts=True)[1][0] / np.unique(y_train, return_counts=True)[1][1]
#     )
    

val_evals = []
aucs = []
f1s = []

for fidx in range(5):
    X_train = np.load('/work/zg78/som_folds/CSF_X_train_{}.npy'.format(fidx))
    y_train = np.load('/work/zg78/som_folds/CSF_y_train_{}.npy'.format(fidx))
    X_val = np.load('/work/zg78/som_folds/CSF_X_val_{}.npy'.format(fidx))
    y_val = np.load('/work/zg78/som_folds/CSF_y_val_{}.npy'.format(fidx))
        
    std_scaler = StandardScaler()
    std_scaler.fit(X_train)
    X_train = std_scaler.transform(X_train)
    X_val = std_scaler.transform(X_val)
    selector = RFE(clf, n_features_to_select=8, step=1)
#     clf.fit(X_train, y_train)
    selector = selector.fit(X_train, y_train)

    train_acc = metrics.accuracy_score(y_train, selector.predict(X_train))
    val_acc = metrics.accuracy_score(y_val, selector.predict(X_val))
    val_proba = selector.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    f1 = metrics.f1_score(y_val, selector.predict(X_val))
    val_evals.append(val_acc)
    aucs.append(auc)
    f1s.append(f1)

    print(train_acc, val_acc, auc, f1, '\n')
    
print(sum(val_evals) / len(val_evals))
print(sum(aucs) / len(aucs))
print(sum(f1s) / len(f1s), np.std(f1s))

0.7833333333333333 0.6875 0.8125 0.6153846153846153 

0.8032786885245902 0.6 0.46428571428571425 0.5714285714285714 

0.7049180327868853 0.6 0.7 0.4000000000000001 

0.7704918032786885 0.7333333333333333 0.7000000000000001 0.6 

0.8032786885245902 0.7333333333333333 0.78 0.6 

0.6708333333333334
0.6913571428571428
0.5573626373626374 0.0799540986539088
